In [1]:
import requests 
import time
import sys
import pandas as pd
import numpy as np
from config import stocks_api

In [2]:
nasdaq = pd.read_csv("Resources/nasdaq_screener_1650145895697.csv") # Read Nasdaq csv
notdaq = pd.read_csv("Resources/not_nasdaq.csv") # Elements that are in Nasdaq, continually appended 

nasdaq_tickers = nasdaq["Symbol"] # Create list of unique nasdaq tickers
not_nasdaq_tickers = notdaq["0"].tolist() # Create list of not nasdaq tickers
nasdaq_tickers = [ticker for ticker in nasdaq_tickers # Remove not nasdaq tickers from list
                  if(ticker not in not_nasdaq_tickers)]

nasdaq_data = [] # Empty list to hold our dictionary
nasdaq_null = [] # List to hold null values 

time_span = input("Select a Time Span (minute, hour, day, week, month, quarter, year): ")
multiplier = input("Multiple of Time Span: ")
start_date = input("Start Date (year-mo-da): ")
end_date = input("End Date (year-mo-da): ")

with open('inputs.py', 'w') as f: # Write variables to the file inputs.py
    sys.stdout = f 
    print(f'time_span = "{time_span}" \nmultiplier = "{multiplier}" \nstart_date = "{start_date}" \nend_date = "{end_date}" \n')
    sys.stdout = sys.stdout 

# Create counters.
record_count = 1
set_count = 1


Select a Time Span (minute, hour, day, week, month, quarter, year): day
Multiple of Time Span: 1
Start Date (year-mo-da): 2022-03-24
End Date (year-mo-da): 2022-03-30


In [ ]:
for ticker in nasdaq_tickers:
 
    try:
        url = f"https://api.polygon.io/v2/aggs/ticker/{ticker}/range/{multiplier}/{time_span}/{start_date}/{end_date}?adjusted=true&sort=asc&limit=120&apiKey=" + stocks_api
        ticker_request = requests.get(url)    
        ticker_data = ticker_request.json()
        
        volume = []
        volume_weighted_average = []
        open_price = []
        close_price = []
        max_price = []
        lowest_price = []
        unix_time = []
        number_of_transactions = []

        for j in range(len(ticker_data["results"])):
            volume.append(ticker_data["results"][j]["v"])
            volume_weighted_average.append(ticker_data["results"][j]["vw"])
            open_price.append(ticker_data["results"][j]["o"])
            close_price.append(ticker_data["results"][j]["c"])
            max_price.append(ticker_data["results"][j]["h"])
            lowest_price.append(ticker_data["results"][j]["l"])
            unix_time.append(ticker_data["results"][j]["t"])
            number_of_transactions.append(ticker_data["results"][j]["n"])
    
    
    
        nasdaq_data.append({"Tickers": ticker, 
                            "Volume": volume,
                            "Volume Weighted Average": volume_weighted_average,
                            "Open Price": open_price,
                            "Close Price": close_price,
                            "Max Price": max_price,
                            "Lowest Price": lowest_price,
                            "Unix Time": unix_time,
                            "Number of Transactions": number_of_transactions})

    except:
        print(f"The Ticker, " + ticker + " is unavailible")
        nasdaq_null.append(ticker)
        nasdaq_null_df = pd.DataFrame(nasdaq_null)
        nasdaq_null_df.to_csv("Resources/not_nasdaq_new.csv", index_label="Tickers")
               
        pass
    
    time.sleep(12) # Free subscription only allows for 5 requests a minute
       

In [ ]:
nasdaq_df = pd.DataFrame(nasdaq_data)
output_data_file = "Resources/Nasdaq_data.csv"

nasdaq_df.to_csv(output_data_file, index_label="Ticker number")
nasdaq_df